In [1]:
import pandas as pd
import numpy as np

# Reshaping and pivot tables
## Reshaping by pivoting DataFrame objects

In [2]:
import pandas._testing as tm

def unpivot(frame):
    N, K = frame.shape
    data = {
        "value": frame.to_numpy().ravel("F"),
        "variable": np.asarray(frame.columns).repeat(N),
        "date": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["date", "variable", "value"])


In [3]:
df = unpivot(tm.makeTimeDataFrame(3))

df

,date,variable,value
0,2000-01-03,A,-1.105214
1,2000-01-04,A,-0.272782
2,2000-01-05,A,-1.187578
3,2000-01-03,B,0.621970
4,2000-01-04,B,-0.187670
5,2000-01-05,B,-1.776844
6,2000-01-03,C,-0.129404
7,2000-01-04,C,0.533256
8,2000-01-05,C,-0.090862
9,2000-01-03,D,1.610582


In [4]:
filtered = df[df["variable"] == "A"]

filtered

,date,variable,value
0,2000-01-03,A,-1.105214
1,2000-01-04,A,-0.272782
2,2000-01-05,A,-1.187578


In [5]:
tm.makeTimeDataFrame(3)

,A,B,C,D
2000-01-03,0.895335,-0.704378,-1.219671,0.929063
2000-01-04,-0.353766,-0.494940,-2.342713,0.153808
2000-01-05,-0.616584,0.181193,-0.755129,0.694561


In [6]:
df

,date,variable,value
0,2000-01-03,A,-1.105214
1,2000-01-04,A,-0.272782
2,2000-01-05,A,-1.187578
3,2000-01-03,B,0.621970
4,2000-01-04,B,-0.187670
5,2000-01-05,B,-1.776844
6,2000-01-03,C,-0.129404
7,2000-01-04,C,0.533256
8,2000-01-05,C,-0.090862
9,2000-01-03,D,1.610582


In [7]:
help(df.pivot)

Help on method pivot in module pandas.core.frame:

pivot(*, columns, index=typing.Literal[<no_default>], values=typing.Literal[<no_default>]) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Return reshaped DataFrame organized by given index / column values.
    
    Reshape data (produce a "pivot" table) based on column values. Uses
    unique values from specified `index` / `columns` to form axes of the
    resulting DataFrame. This function does not support data
    aggregation, multiple values will result in a MultiIndex in the
    columns. See the :ref:`User Guide <reshaping>` for more on reshaping.
    
    Parameters
    ----------
    columns : str or object or a list of str
        Column to use to make new frame's columns.
    
        .. versionchanged:: 1.1.0
           Also accept list of columns names.
    
    index : str or object or a list of str, optional
        Column to use to make new frame's index. If not given, uses existing index.
    
        

In [9]:
df.pivot(columns='variable', index='date', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,-1.105214,0.621970,-0.129404,1.610582
2000-01-04,-0.272782,-0.187670,0.533256,-0.237625
2000-01-05,-1.187578,-1.776844,-0.090862,1.289117


In [10]:
df["value2"] = df["value"] * 2

pivoted = df.pivot(index="date", columns="variable")

pivoted

value                                  value2             
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -1.105214  0.621970 -0.129404  1.610582 -2.210428  1.243940  \
2000-01-04 -0.272782 -0.187670  0.533256 -0.237625 -0.545564 -0.375340   
2000-01-05 -1.187578 -1.776844 -0.090862  1.289117 -2.375157 -3.553687   

                                
variable           C         D  
date                            
2000-01-03 -0.258808  3.221164  
2000-01-04  1.066512 -0.475251  
2000-01-05 -0.181725  2.578235

# Reshaping by stacking and unstacking

In [15]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)

In [16]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [18]:
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [19]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])

df2 = df[:4]

df2

A         B
first second                    
bar   one     0.090692  0.965208
      two     1.113235 -0.542412
baz   one    -0.431746 -1.768195
      two    -0.851739  1.900908

In [31]:
stacked = df2.stack()

stacked

first  second   
bar    one     A    0.090692
               B    0.965208
       two     A    1.113235
               B   -0.542412
baz    one     A   -0.431746
               B   -1.768195
       two     A   -0.851739
               B    1.900908
dtype: float64

In [35]:
unstacked = stacked.unstack(0)
unstacked

first          bar       baz
second                      
one    A  0.090692 -0.431746
       B  0.965208 -1.768195
two    A  1.113235 -0.851739
       B -0.542412  1.900908

In [36]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.090692  1.113235
      B  0.965208 -0.542412
baz   A -0.431746 -0.851739
      B -1.768195  1.900908

In [37]:
stacked.unstack(2)

A         B
first second                    
bar   one     0.090692  0.965208
      two     1.113235 -0.542412
baz   one    -0.431746 -1.768195
      two    -0.851739  1.900908

In [38]:
stacked.unstack("second")


second        one       two
first                      
bar   A  0.090692  1.113235
      B  0.965208 -0.542412
baz   A -0.431746 -0.851739
      B -1.768195  1.900908

## Multiple levels

In [39]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat", "long"),
        ("B", "cat", "long"),
        ("A", "dog", "short"),
        ("B", "dog", "short"),
    ],
    names=["exp", "animal", "hair_length"],
)


In [40]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)

df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,0.492360,0.330905,0.485340,2.258194
1,0.539350,-1.498203,-0.925922,-0.463165
2,-1.044514,-1.469075,-0.955064,0.543133
3,0.290365,-0.565860,-0.180701,-0.258899


In [48]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long         0.492360  0.330905
  dog    short        0.485340  2.258194
1 cat    long         0.539350 -1.498203
  dog    short       -0.925922 -0.463165
2 cat    long        -1.044514 -1.469075
  dog    short       -0.955064  0.543133
3 cat    long         0.290365 -0.565860
  dog    short       -0.180701 -0.258899

In [49]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long         0.492360  0.330905
  dog    short        0.485340  2.258194
1 cat    long         0.539350 -1.498203
  dog    short       -0.925922 -0.463165
2 cat    long        -1.044514 -1.469075
  dog    short       -0.955064  0.543133
3 cat    long         0.290365 -0.565860
  dog    short       -0.180701 -0.258899

## Missing data

In [50]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat"),
        ("B", "dog"),
        ("B", "cat"),
        ("A", "dog"),
    ],
    names=["exp", "animal"],
)


index = pd.MultiIndex.from_product(
    [("bar", "baz", "foo", "qux"), ("one", "two")], names=["first", "second"]
)


df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

df2 = df.iloc[[0, 1, 2, 4, 5, 7]]

df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.071538 -0.021112 -1.605439  0.916700
      two    -0.725512  0.939792 -0.329861 -0.361862
baz   one    -0.879042 -1.636553  0.746800 -0.604879
foo   one     0.654284 -0.226254  0.776808  1.633817
      two     0.845302 -0.441618 -0.328818  0.213529
qux   two    -0.397921  0.359550 -1.779218  0.395613

In [51]:
df2.stack("exp")

animal                 cat       dog
first second exp                    
bar   one    A    1.071538  0.916700
             B   -1.605439 -0.021112
      two    A   -0.725512 -0.361862
             B   -0.329861  0.939792
baz   one    A   -0.879042 -0.604879
             B    0.746800 -1.636553
foo   one    A    0.654284  1.633817
             B    0.776808 -0.226254
      two    A    0.845302  0.213529
             B   -0.328818 -0.441618
qux   two    A   -0.397921  0.395613
             B   -1.779218  0.359550

In [53]:
df2.stack("animal")

exp                         A         B
first second animal                    
bar   one    cat     1.071538 -1.605439
             dog     0.916700 -0.021112
      two    cat    -0.725512 -0.329861
             dog    -0.361862  0.939792
baz   one    cat    -0.879042  0.746800
             dog    -0.604879 -1.636553
foo   one    cat     0.654284  0.776808
             dog     1.633817 -0.226254
      two    cat     0.845302 -0.328818
             dog     0.213529 -0.441618
qux   two    cat    -0.397921 -1.779218
             dog     0.395613  0.359550

In [55]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.021112 -1.605439
      two     0.939792 -0.329861
foo   one    -0.226254  0.776808
qux   two     0.359550 -1.779218

In [56]:
df3.unstack()


exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.021112  0.939792 -1.605439 -0.329861
foo    -0.226254       NaN  0.776808       NaN
qux          NaN  0.359550       NaN -1.779218

In [57]:
df3.unstack(fill_value=-1e9)


exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar    -2.111153e-02  9.397919e-01 -1.605439e+00 -3.298612e-01
foo    -2.262538e-01 -1.000000e+09  7.768077e-01 -1.000000e+09
qux    -1.000000e+09  3.595497e-01 -1.000000e+09 -1.779218e+00

## With a MultiIndex

In [58]:
df[:3].unstack(0)


exp            A                   B                                     A   
animal       cat                 dog                 cat               dog   
first        bar       baz       bar       baz       bar     baz       bar   
second                                                                       
one     1.071538 -0.879042 -0.021112 -1.636553 -1.605439  0.7468  0.916700  \
two    -0.725512       NaN  0.939792       NaN -0.329861     NaN -0.361862   

exp               
animal            
first        baz  
second            
one    -0.604879  
two          NaN

In [59]:
df2.unstack(1)


exp            A                   B                                       A   
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar     1.071538 -0.725512 -0.021112  0.939792 -1.605439 -0.329861  0.916700  \
baz    -0.879042       NaN -1.636553       NaN  0.746800       NaN -0.604879   
foo     0.654284  0.845302 -0.226254 -0.441618  0.776808 -0.328818  1.633817   
qux          NaN -0.397921       NaN  0.359550       NaN -1.779218       NaN   

exp               
animal            
second       two  
first             
bar    -0.361862  
baz          NaN  
foo     0.213529  
qux     0.395613

# Reshaping by melt

In [64]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)

cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [62]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [65]:
cheese.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [67]:
index = pd.MultiIndex.from_tuples([("person", "A"), ("person", "B")])

cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    },
    index=index,
)

cheese

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [68]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [69]:
cheese.melt(id_vars=["first", "last"], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

In [70]:
dft = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
    }
)

dft["id"] = dft.index

dft

pd.wide_to_long(dft, ["A", "B"], i="id", j="year")


,,X,A,B
id,year,,,
0,1970,0.597496,a,2.5
1,1970,1.312463,b,1.2
2,1970,-0.125278,c,0.7
0,1980,0.597496,d,3.2
1,1980,1.312463,e,1.3
2,1980,-0.125278,f,0.1


# Combining with stats and GroupBy

In [71]:
df


exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.071538 -0.021112 -1.605439  0.916700
      two    -0.725512  0.939792 -0.329861 -0.361862
baz   one    -0.879042 -1.636553  0.746800 -0.604879
      two     1.241926 -0.299117  0.646043 -0.233834
foo   one     0.654284 -0.226254  0.776808  1.633817
      two     0.845302 -0.441618 -0.328818  0.213529
qux   one     0.392982  1.504408  0.970314  0.663231
      two    -0.397921  0.359550 -1.779218  0.395613

In [77]:
df.stack().mean(1).unstack()


animal             cat       dog
first second                    
bar   one    -0.266951  0.447794
      two    -0.527686  0.288965
baz   one    -0.066121 -1.120716
      two     0.943984 -0.266475
foo   one     0.715546  0.703782
      two     0.258242 -0.114045
qux   one     0.681648  1.083820
      two    -1.088570  0.377581

In [79]:
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one    -0.266951  0.447794
      two    -0.527686  0.288965
baz   one    -0.066121 -1.120716
      two     0.943984 -0.266475
foo   one     0.715546  0.703782
      two     0.258242 -0.114045
qux   one     0.681648  1.083820
      two    -1.088570  0.377581

In [80]:
df.mean().unstack(0)


exp,A,B
animal,,
cat,0.275445,-0.112922
dog,0.327789,0.022387


In [81]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.071538 -0.021112 -1.605439  0.916700
      two    -0.725512  0.939792 -0.329861 -0.361862
baz   one    -0.879042 -1.636553  0.746800 -0.604879
      two     1.241926 -0.299117  0.646043 -0.233834
foo   one     0.654284 -0.226254  0.776808  1.633817
      two     0.845302 -0.441618 -0.328818  0.213529
qux   one     0.392982  1.504408  0.970314  0.663231
      two    -0.397921  0.359550 -1.779218  0.395613

In [83]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,0.481079,0.063622
two,0.122155,-0.154156


# Pivot tables

In [85]:
import datetime

df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)]
        + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
    }
)
df

,A,B,C,D,E,F
0,one,A,foo,1.097910,0.417138,2013-01-01
1,one,B,foo,0.017962,0.536322,2013-02-01
2,two,C,foo,0.201249,-0.166473,2013-03-01
3,three,A,bar,-0.993570,-0.107658,2013-04-01
4,one,B,bar,-1.264326,-1.286630,2013-05-01
5,one,C,bar,0.380259,-1.039705,2013-06-01
6,two,A,foo,-1.100849,-0.029464,2013-07-01
7,three,B,foo,0.832216,-1.524001,2013-08-01
8,one,C,foo,-0.235487,1.078809,2013-09-01
9,one,A,bar,-2.284530,-0.465508,2013-10-01


In [89]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"], aggfunc=np.mean)

C             bar       foo
A     B                    
one   A -1.758627  0.340831
      B  0.198247 -0.344204
      C -0.075271 -0.839947
three A -1.298440       NaN
      B       NaN  0.037036
      C -0.509952       NaN
two   A       NaN -1.024076
      B  0.119611       NaN
      C       NaN  0.831221

In [87]:
pd.pivot_table(df, values="D", index=["B"], columns=["A", "C"], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A -3.517254  0.681663 -2.596881       NaN       NaN -2.048151
B  0.396495 -0.688408       NaN  0.074072  0.239223       NaN
C -0.150541 -1.679893 -1.019904       NaN       NaN  1.662442

In [90]:
pd.pivot_table(
    df, values=["D", "E"],
    index=["B"],
    columns=["A", "C"],
    aggfunc=np.sum,
)

D                                                           E   
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A -3.517254  0.681663 -2.596881       NaN       NaN -2.048151  0.459215  \
B  0.396495 -0.688408       NaN  0.074072  0.239223       NaN -0.633130   
C -0.150541 -1.679893 -1.019904       NaN       NaN  1.662442 -0.006557   

                                                    
A               three                two            
C       foo       bar      foo       bar       foo  
B                                                   
A  1.779363  1.054476      NaN       NaN  0.660781  
B  0.536890       NaN -1.35935  1.458612       NaN  
C  0.898304 -1.112505      NaN       NaN -0.060788

In [91]:
pd.pivot_table(df[["A", "B", "C", "D", "E"]], index=["A", "B"], columns=["C"])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -1.758627  0.340831  0.229608  0.889681
      B  0.198247 -0.344204 -0.316565  0.268445
      C -0.075271 -0.839947 -0.003279  0.449152
three A -1.298440       NaN  0.527238       NaN
      B       NaN  0.037036       NaN -0.679675
      C -0.509952       NaN -0.556252       NaN
two   A       NaN -1.024076       NaN  0.330390
      B  0.119611       NaN  0.729306       NaN
      C       NaN  0.831221       NaN -0.030394

In [92]:
pd.pivot_table(df, values="D", index=pd.Grouper(freq="M", key="F"), columns="C")


C,bar,foo
F,,
2013-01-31,NaN,0.340831
2013-02-28,NaN,-0.344204
2013-03-31,NaN,0.831221
2013-04-30,-1.298440,NaN
2013-05-31,0.198247,NaN
2013-06-30,-0.075271,NaN
2013-07-31,NaN,-1.024076
2013-08-31,NaN,0.037036
2013-09-30,NaN,-0.839947


In [93]:
table = pd.pivot_table(df, index=["A", "B"], columns=["C"], values=["D", "E"])

print(table.to_string(na_rep=""))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -1.758627  0.340831  0.229608  0.889681
      B  0.198247 -0.344204 -0.316565  0.268445
      C -0.075271 -0.839947 -0.003279  0.449152
three A -1.298440            0.527238          
      B            0.037036           -0.679675
      C -0.509952           -0.556252          
two   A           -1.024076            0.330390
      B  0.119611            0.729306          
      C            0.831221           -0.030394


## Adding margins

In [94]:
table = df.pivot_table(
    index=["A", "B"],
    columns="C",
    values=["D", "E"],
    margins=True,
    aggfunc=np.std
)


In [95]:
table


D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.743740  1.070671  1.426792  0.983042  0.668278  0.784998
      B  2.068392  0.512180  1.269491  1.371879  0.378835  0.888408
      C  0.644217  0.854834  0.759492  1.465728  0.890470  1.024042
three A  0.431152       NaN  0.431152  0.897878       NaN  0.897878
      B       NaN  1.124554  1.124554       NaN  1.194057  1.194057
      C  0.101767       NaN  0.101767  0.625199       NaN  0.625199
two   A       NaN  0.108573  0.108573       NaN  0.508911  0.508911
      B  0.169096       NaN  0.169096  1.313964       NaN  1.313964
      C       NaN  0.890914  0.890914       NaN  0.192445  0.192445
All      1.043743  0.916473  0.960115  0.969571  0.730956  0.823649

In [96]:
table.stack()


D         E
A     B C                      
one   A All  1.426792  0.784998
        bar  0.743740  0.983042
        foo  1.070671  0.668278
      B All  1.269491  0.888408
        bar  2.068392  1.371879
        foo  0.512180  0.378835
      C All  0.759492  1.024042
        bar  0.644217  1.465728
        foo  0.854834  0.890470
three A All  0.431152  0.897878
        bar  0.431152  0.897878
      B All  1.124554  1.194057
        foo  1.124554  1.194057
      C All  0.101767  0.625199
        bar  0.101767  0.625199
two   A All  0.108573  0.508911
        foo  0.108573  0.508911
      B All  0.169096  1.313964
        bar  0.169096  1.313964
      C All  0.890914  0.192445
        foo  0.890914  0.192445
All     All  0.960115  0.823649
        bar  1.043743  0.969571
        foo  0.916473  0.730956

# Cross tabulations

In [97]:
foo, bar, dull, shiny, one, two = "foo", "bar", "dull", "shiny", "one", "two"

a = np.array([foo, foo, bar, bar, foo, foo], dtype=object)

b = np.array([one, one, two, one, two, one], dtype=object)

c = np.array([dull, dull, shiny, dull, dull, shiny], dtype=object)

pd.crosstab(a, [b, c], rownames=["a"], colnames=["b", "c"])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     0    0     1
foo    2     1    1     0

In [98]:
df = pd.DataFrame(
    {"A": [1, 2, 2, 2, 2], "B": [3, 3, 4, 4, 4], "C": [1, 1, np.nan, 1, 1]}
)


df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [100]:
pd.crosstab(df['A'], df['B'])

B,3,4
A,,
1,1,0
2,1,3


In [101]:
foo = pd.Categorical(["a", "b"], categories=["a", "b", "c"])

bar = pd.Categorical(["d", "e"], categories=["d", "e", "f"])

pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


In [102]:
pd.crosstab(foo, bar, dropna=False)

col_0,d,e,f
row_0,,,
a,1,0,0
b,0,1,0
c,0,0,0


## Normalization

In [104]:
pd.crosstab(df["A"], df["B"], normalize=True)


B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


In [105]:
pd.crosstab(df["A"], df["B"], normalize="columns")


B,3,4
A,,
1,0.5,0.0
2,0.5,1.0


In [106]:
pd.crosstab(df["A"], df["B"], values=df["C"], aggfunc=np.sum)


B,3,4
A,,
1,1.0,NaN
2,1.0,2.0


## Add margins

In [108]:
pd.crosstab(
    df["A"], df["B"], values=df["C"], aggfunc=np.sum, normalize=True
)

B,3,4
A,,
1,0.25,0.0
2,0.25,0.5


In [107]:
pd.crosstab(
    df["A"], df["B"], values=df["C"], aggfunc=np.sum, normalize=True, margins=True
)

B,3,4,All
A,,,
1,0.25,0.0,0.25
2,0.25,0.5,0.75
All,0.50,0.5,1.00


# Tiling

In [109]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])


In [111]:
c = pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60.0], (43.333, 60.0]]
Categories (3, interval[float64, right]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

In [112]:
c = pd.cut(ages, bins=[0, 18, 35, 70])


# Computing indicator / dummy variables

In [113]:
df = pd.DataFrame({"key": list("bbacab"), "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [114]:
pd.get_dummies(df["key"])


,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [115]:
dummies = pd.get_dummies(df["key"], prefix="key")


In [116]:
df[["data1"]].join(dummies)


,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


In [117]:
values = np.random.randn(10)

values

array([-0.389223  , -1.36618669,  0.52115759, -1.43627201,  0.51720321,
       -0.64286602, -0.4464513 ,  0.74944354,  0.72860642,  0.51881968])

In [118]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]


In [119]:
pd.get_dummies(pd.cut(values, bins))


,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,True,False,False
3,False,False,False,False,False
4,False,False,True,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,True,False,False


In [120]:
df = pd.DataFrame({"A": ["a", "b", "a"], "B": ["c", "c", "b"], "C": [1, 2, 3]})

pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,True,False,False,True
1,2,False,True,False,True
2,3,True,False,True,False


In [121]:
simple = pd.get_dummies(df, prefix="new_prefix")
simple

,C,new_prefix_a,new_prefix_b,new_prefix_b,new_prefix_c
0,1,True,False,False,True
1,2,False,True,False,True
2,3,True,False,True,False


In [123]:
s = pd.Series(list("abcaa"))
pd.get_dummies(s)


,a,b,c
0,True,False,False
1,False,True,False
2,False,False,True
3,True,False,False
4,True,False,False


# Factorizing values

In [124]:
x = pd.Series(["A", "A", np.nan, "B", 3.14, np.inf])
x

0       A
1       A
2     NaN
3       B
4    3.14
5     inf
dtype: object

In [126]:
labels, uniques = pd.factorize(x)
labels

array([ 0,  0, -1,  1,  2,  3])

In [127]:
uniques

Index(['A', 'B', 3.14, inf], dtype='object')

# Examples

In [128]:
np.random.seed([3, 1415])

n = 20

cols = np.array(["key", "row", "item", "col"])

df = cols + pd.DataFrame(
    (np.random.randint(5, size=(n, 4)) // [2, 1, 2, 1]).astype(str)
)


df.columns = cols

df = df.join(pd.DataFrame(np.random.rand(n, 2).round(2)).add_prefix("val"))

df


,key,row,item,col,val0,val1
0,key0,row3,item1,col3,0.81,0.04
1,key1,row2,item1,col2,0.44,0.07
2,key1,row0,item1,col0,0.77,0.01
3,key0,row4,item0,col2,0.15,0.59
4,key1,row0,item2,col1,0.81,0.64
5,key1,row2,item2,col4,0.13,0.88
6,key2,row4,item1,col3,0.88,0.39
7,key1,row4,item1,col1,0.10,0.07
8,key1,row0,item2,col4,0.65,0.02
9,key1,row2,item0,col2,0.35,0.61


In [150]:
df = pd.DataFrame(data=np.hstack([np.random.randint(0,2,(14,5)),
                  np.random.choice(['foo', 'bar'], size=(14,1))]),
                 columns=list('abcdef'))

df

,a,b,c,d,e,f
0,1,1,1,0,0,foo
1,1,1,1,1,0,bar
2,1,1,1,0,1,bar
3,1,0,1,0,1,bar
4,0,1,0,0,1,foo
5,0,0,0,0,1,foo
6,0,0,1,1,1,foo
7,1,0,0,1,0,foo
8,1,1,0,0,1,foo
9,0,0,1,0,1,foo


In [151]:
pd.pivot_table(df, index=['a','f'], values='d', columns='e', aggfunc=[np.mean, np.sum])

mean       sum       
e        0     1   0      1
a f                        
0 bar  1.0   NaN   1    NaN
  foo  0.0  20.0   0  00100
1 bar  1.0   0.0   1     00
  foo  0.5   0.5  01     01

In [145]:
help(pd.pivot_table)

Help on function pivot_table in module pandas.core.reshape.pivot:

pivot_table(data: 'DataFrame', values=None, index=None, columns=None, aggfunc: 'AggFuncType' = 'mean', fill_value=None, margins: 'bool' = False, dropna: 'bool' = True, margins_name: 'Hashable' = 'All', observed: 'bool' = False, sort: 'bool' = True) -> 'DataFrame'
    Create a spreadsheet-style pivot table as a DataFrame.
    
    The levels in the pivot table will be stored in MultiIndex objects
    (hierarchical indexes) on the index and columns of the result DataFrame.
    
    Parameters
    ----------
    data : DataFrame
    values : list-like or scalar, optional
        Column or columns to aggregate.
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed,
        it is being used as the same manner a

In [142]:
np.hstack([np.random.randint(0,5,(4,5)),
                  np.random.choice(['foo', 'bar'], size=(4,1))])

array([['2', '1', '3', '4', '0', 'bar'],
       ['4', '4', '3', '2', '1', 'bar'],
       ['0', '2', '0', '3', '1', 'foo'],
       ['3', '1', '4', '2', '0', 'foo']], dtype='<U21')